# 문서 분류(Document Classification)

## 데이터 준비

* 문서 분류에 필요한 데이터는 `scikit-learn`이 제공하는 20개의 주제를 가지는 뉴스그룹 데이터를 사용
* 텍스트는 `CounterVectorizer`를 거쳐 DTM 행렬로 변환
* DTM 행렬은 문서에 등장하는 단어들을 빈도 수 별로 표현한 행렬


In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

news = fetch_20newsgroups()

x = news.data
y = news.target

cv = CountVectorizer()
x = cv.fit_transform(x) # countvector로 계산한번하고 x에 다시 넣는거지

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(7919, 130107) (7919,) (3395, 130107) (3395,)


In [2]:
print(x_train[0])

# 0번째 문서에서 56979 
# (0문서의 ,56979의 인덱스의 단어가) 1번 등장


  (0, 56979)	1
  (0, 111322)	1
  (0, 68532)	1
  (0, 114731)	2
  (0, 90379)	1
  (0, 89362)	7
  (0, 76032)	1
  (0, 123292)	2
  (0, 90774)	1
  (0, 89860)	2
  (0, 114455)	12
  (0, 115475)	6
  (0, 32311)	1
  (0, 66608)	4
  (0, 27436)	2
  (0, 90252)	1
  (0, 62221)	2
  (0, 35983)	4
  (0, 123305)	1
  (0, 56283)	2
  (0, 114494)	2
  (0, 124616)	2
  (0, 28146)	6
  (0, 29241)	1
  (0, 26605)	1
  :	:
  (0, 76246)	1
  (0, 54093)	1
  (0, 53628)	1
  (0, 67800)	1
  (0, 34795)	2
  (0, 45254)	1
  (0, 66701)	1
  (0, 41799)	1
  (0, 26190)	1
  (0, 28853)	1
  (0, 106272)	2
  (0, 80697)	1
  (0, 45255)	1
  (0, 63963)	1
  (0, 110242)	2
  (0, 96161)	2
  (0, 42958)	1
  (0, 41490)	1
  (0, 52910)	1
  (0, 53561)	1
  (0, 92677)	1
  (0, 60898)	1
  (0, 86551)	1
  (0, 102126)	1
  (0, 12257)	1


## scikit-learn을 이용한 문서 분류

In [3]:
from sklearn.metrics import accuracy_score

### Logistic Regression

* Logistic Regression은 특성상 다중 분류에는 적합하지 않음

In [5]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(x_train, y_train)    # 학습하고
pred = LR.predict(x_test)   # 예측하고
acc = accuracy_score(pred, y_test)  # 그 정확도를 실제값이랑 비교해서 얻어내야지
print(acc)

# 0.86이라..
# 이건 다중이 아니니까 제외하고

0.8639175257731959


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Support Vector Machine

In [6]:
from sklearn import svm

SVM = svm.SVC(kernel = 'linear')
SVM.fit(x_train, y_train)
pred = SVM.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)
# 그래도 나름 분류를 잘하느 svm 소프트 백터 머신. 한번 봐볼까
# 0.82 %라..

0.825920471281296


In [8]:
x_train[:3]

<3x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 693 stored elements in Compressed Sparse Row format>

### Naive Bayes

* 베이즈 정리를 적용한 확률적 분류 알고리즘
* 모든 특성들이 독립임을 가정(naive 가정)
* 입력 특성에 따라 3개의 분류기 존재


> * 가우시안 나이브 베이즈 분류기
> * 베르누이 나이브 베이즈 분류기
> * 다항 나이브 베이즈 분류기




#### DTM을 이용한 Naive Bayes

In [10]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(x_train, y_train)
pred = NB.predict(x_test)
acc = accuracy_score(pred, y_test)
acc
# 0.81

0.8138438880706922

#### tf-idf를 이용한 정확도 향상

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
x_train_tf = tfidf.fit_transform(x_train)
x_test_tf = tfidf.fit_transform(x_test)

NB.fit(x_train_tf, y_train)
pred = NB.predict(x_test_tf)
acc = accuracy_score(pred,y_test)
acc

# 오 3% 올랐네

0.8300441826215023

### Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(x_train, y_train)
pred = DT.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

#현재 데이터로서는 규칙을 찾기 어려우니까
# 문서랑 index와 index사이의 데이터로만 되어있으니까 당연히 규칙 찾기 어렵지..

0.6164948453608248


### XGBoost

In [13]:
# 앙상블
# 분류에 있어 다른 알고리즘보다 좋은 예측성능을 보임
# GBM기준이지만 특유의 느린 수행시간과 과적합 규제 부재등의 문제를 해결
# 별령 cpu 환경에서 빠르게 학습 가능

from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators = 30, learning_rate = 0.05, max_depth = 3, )
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

# 아니뭐..그래도 트리기반중에서는.. 10% 더 놓게 나오긴 했다만..

0.7116347569955818


##교차 검증

* 일반 검증은 학습 데이터가 테스트 데이터로 사용되지 않음
* 교차 검증은 데이터를 n개의 집합으로 나누어 정확도를 계산해 학습 데이터로 사용된 데이터도 테스트 데이터로 사용
* 교차 검증을 사용하면 일반 검증보다 모델의 일반화가 잘 되어 있는지 평가 가능
* 앞서 구성한 나이브 베이즈 모델을 교차 검증

In [15]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(NB, x, y, cv = 5)  # 5개의 집합으로 구분할게
print(scores, scores.mean())

# 평균 0.83 성능이 어느정도 높게 나오네


[0.83870968 0.83826779 0.82368537 0.83031374 0.83642794] 0.833480903927519


* 교차 검증을 통해 일반 검증보다 좀 더 일반화된 모델 생성 가능
* 교차 검증은 일반 검증에 비해 n번 검증을 해 비용이 더 많이 소요

## 정밀도와 재현률 

* 정밀도(precision)는 양성 클래스(정답)으로 예측한 샘플이 양성 클래스일 확률을 의미
* 모델이 얼마나 양성 클래스를 잘 예측하는지를 나타냄
* 재현률(recall)은 양성 클래스인 샘플에서 모델이 양성 클래스로 예측한 샘플 비율을 의미하며, 모델이 얼마나 실제 상황을 재현하는지를 나타냄
* 정밀도와 재현율의 가중조화평균인 F1-score라는 지표는 정확도에 비해 더 효과적인 모델 분석 지표로 알려져 있음
* 직접 계산할 수도 있으나, scikit-learn은 이를 편리하게 계산해주는 함수를 제공

* 다중 클래스 분류 문제에서 정밀도와 재현률을 계산할 때는 클래스간의 지표를 어떻게 합칠지 지정 필요

  * None - 클래스간 지표를 합치지 말고 그대로 출력
  * micro - 정밀도와 재현률이 같음, 이로 인해 f1-score도 정밀도, 재현률과 동일
  * macro - 클래스간 지표를 단순 평균한 값
  * weighted - 클래스간 지표를 가중 평균한 값

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(pred, y_test, average = 'micro')
recall = recall_score(pred, y_test, average = 'micro')
f1 = f1_score(pred, y_test, average = 'micro')


print( precision, recall, f1)   # micro로 했기때문에 다 같은 값으로 나옴

# accuracy랑 precision이랑 다름// 비율과 확률

0.7116347569955818 0.7116347569955818 0.7116347569955818


In [18]:
precision = precision_score(pred, y_test, average = 'macro')
recall = recall_score(pred, y_test, average = 'macro')
f1 = f1_score(pred, y_test, average = 'macro')


print( precision, recall, f1) 
# 이번엔 다 제각가이고

0.7062145232753229 0.7450917437339617 0.7173265215507635


## 그리드 검색을 이용한 파라미터 최적화

* 그리드 검색을 사용하면 분류기에 사용하는 파라미터 최적화 가능
* 그리드 검색을 통해 앞서 구성한 나이브 베이즈 모델의 'alpha' 파라미터를 최적화시키는 예제

* `estimator`: 사용 모델 객체     
* `param_grid`: 사용 객체:지정 파라미터 리스트로 구성된 딕셔너리    
* `scoring`: 최적화하고자 하는 성능 지표   
* `cv`: 교차 검증 분할 개수      

In [24]:
from sklearn.model_selection import GridSearchCV
# GS = GridSearchCV(estimator = NB, param_grid = {'alpha' :[0.001, 0.01, 0.1, 1.]}, scoring = 'accuracy', cv = 10)
# GS = GridSearchCV(estimator = NB, param_grid = {'alpha' :[0.001, 0.002, 0.003, 0.004,0.005]}, scoring = 'accuracy', cv = 10)
GS = GridSearchCV(estimator = NB, param_grid = {'alpha' :[0.00013, 0.00012, 0.001, 0.0011]}, scoring = 'accuracy', cv = 10)

GS.fit(x,y)


print(GS.best_score_)
print(GS.best_params_)

# 니베아 방법의 grid를 수정 결과 
# 88퍼 까지 올라갔구만

# 0.001이 가장 좋았으니까 그 근처값으로 다시 바꿔서 해보자


0.891638548081219
{'alpha': 0.00012}
